In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as pl

from sklearn.model_selection import cross_validate # for cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

##############################################
# Funções para converter e reverter a escala 
###

def scale01(x):
    return (x - x.min())/(x.max() - x.min())

def unscale01(x, lower, upper):
    #return (x * (upper - lower)) + lower
    return (x * upper) - ((x - 1.0) * lower)

#############################################

<font size="5">**Reading Data**</font>

In [ ]:
df = pd.read_csv('trainning.data', delimiter=';')

data = np.array(df.values[: , 2:], dtype = float)   # Pandas dtype = object, logo tudo é permitido
(N, d) = data.shape

print(N, 'x', d)